# Introduction to Python in GIS
by: Bo Fethe

Pre-reqs:
+ ArcGIS Pro

## Setup

Out of the box, ESRI gives you a [conda](https://anaconda.org/Esri) python environment that functions similar to Anaconda's base environment. This means that you should not (and in this case, cannot) edit this environment or you risk running into issues later such as with cloned environment dependencies.

The first thing you should do is clone this environment. Open ArcGIS Pro and navigate to `Project` > `Package Manager` and in the `Active Environment` box click on the `gear icon` > `...` next to the default python environment > `Clone`, and then give your new environment a name and path. After it's cloned, activate your new environment by right clicking it and selecting `Activate`. Now that you have this setup, you can install other libraries, update libraries, break it, fix it, ...whatever.  You can easily default back to the base if something goes wrong.

ESRI has 2 important python libraries to be familiar with - `arcpy` for accessing local files and geoprocessing tools and `arcgis` for accessing web layers and cloud-based geoprocessing tools.

In [2]:
# ESRI libraries
import arcgis
import arcpy

# Non-ESRI libraries
import os
import sys

Print the current python version and authenticate yourself through the active portal in ArcGIS Pro. Alternatively, you can pass your `url`, `username` and `password` as parameters.

In [3]:
print('Python version:', sys.version)

gis = arcgis.GIS('Pro')
print('Logged in as:', gis.properties.user.username)

Python version: 3.9.16 [MSC v.1931 64 bit (AMD64)]
Logged in as: Bo.Fethe_fdot1


Create a workspace

In [21]:
arcpy.env.overwriteOutput = True    # Allow overwriting layers
gdbName = 'python-gis' + '.gdb'     # Geodatabase name must include file extension

# Create a new geodatabase.
arcpy.management.CreateFileGDB(out_folder_path='data',
                               out_name=gdbName)

# Set workspace to the new geodatabase
arcpy.env.workspace = os.path.join('data', gdbName)

## Accessing Web Layers

You can access web layers through [arcgis](https://developers.arcgis.com/python/api-reference/). You can either search by name, use the service URL, or by ArcGIS Online item ID as shown below. This layer can be added to an interactive map object and queried as a spatial dataframe. This example is accessing a web layer of Florida's counties with FDOT District ID.

In [22]:
# Define the service URL
districtsID = '0e28ef312008491aa86f90bd9ca7c706'

# Construct the FeatureLayer object 
searchResults = gis.content.search(districtsID, 'Feature Layer')
districtsLayer = searchResults[0].layers[0]

# ## Currently buggy in Pro 3.1 ###
# ## Create a map object and add the layer to it
# mapp = gis.map('Florida')
# mapp.add_layer(districtsLayer)
# mapp

Read the district layer as a spatial dataframe.  There are many similarities between an arcgis spatial dataframe and a pandas dataframe. For example: you can filter the dataframe by passing in a boolean arguement and the indexes where `TRUE` will be returned. The below example filters the districts county layer to only those in FDOT District 7.

In [17]:
# Construct the dataframe and show data type
districtsSDF = districtsLayer.query().sdf
print('districtsSDF data type:', type(districtsSDF))

# Filter and show dataframe
districtsSDF[districtsSDF['FDOTDistr'] == 7]

districtsSDF data type: <class 'pandas.core.frame.DataFrame'>


,FID,NAME,FIRST_FIPS,OBJECTID,FEATURE_AR,FEATURE_LE,FDOTDistr,DistrictString,DEPCODE,DEPCODENUM,DCode,County,FDOTCountyCode,SHAPE
18,19,Pasco,101,51,0.0,0.0,7,Seven,51,51,D7,Pasco,14,"{""rings"": [[[-9207770, 3292091], [-9207695, 32..."
19,20,Pinellas,103,52,0.0,0.0,7,Seven,52,52,D7,Pinellas,15,"{""rings"": [[[-9220234, 3237823], [-9220096, 32..."
37,38,Citrus,017,9,0.0,0.0,7,Seven,9,9,D7,Citrus,02,"{""rings"": [[[-9181660, 3382364], [-9181551, 33..."
54,55,Hernando,053,26,0.0,0.0,7,Seven,27,27,D7,Hernando,08,"{""rings"": [[[-9203518, 3303737], [-9203657, 33..."
56,57,Hillsborough,057,28,0.0,0.0,7,Seven,29,29,D7,Hillsborough,10,"{""rings"": [[[-9212552, 3195753], [-9212586, 31..."


## Local Geoprocessing Tools

You can access the geoprocesing tools you would normally have access to in ArcGIS Pro using the [arcpy](https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/alphabetical-list-of-arcpy-functions.htm) library. This example below is exporting the FDOT Districts layer into our default workspace and then confirming the layers in the geodatabase.

In [18]:
# Export the web layer
arcpy.conversion.ExportFeatures(in_features=districtsSDF,
                                out_features='Districts')

# Make sure the layer exported
print('Local files:', arcpy.ListFeatureClasses(), sep='\n')

Local files:
['Districts']
